In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

pd.set_option("display.precision", 1)

# set a random seed
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)


2023-05-14 16:05:35.141739: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-14 16:05:35.218754: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-14 16:05:35.568953: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-14 16:05:35.571034: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 16:05:36.435809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

## On récupère la donnée

In [2]:
test_customers = pd.read_csv('data/test_customers.csv')
test_locations = pd.read_csv('data/test_locations.csv')

train_customers = pd.read_csv('data/train_customers.csv')
train_locations = pd.read_csv('data/train_locations.csv')

orders = pd.read_csv('data/orders.csv')
vendors = pd.read_csv('data/vendors.csv')


/tmp/ipykernel_21963/3811830009.py:7: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv('data/orders.csv')


In [3]:
print('train_customers shape: ', train_customers.shape)
print('test_customers shape: ', test_customers.shape)
train_customers.head()

train_customers shape:  (34674, 8)
test_customers shape:  (9768, 8)


,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at
0,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23
1,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41
2,S2ALZFL,Male,NaN,0,1,EN,2018-03-14 18:31:43,2018-03-14 18:31:42
3,952DBJQ,Male,NaN,1,1,EN,2018-03-15 19:47:07,2018-03-15 19:47:07
4,1IX6FXS,Male,NaN,1,1,EN,2018-03-15 19:57:01,2018-03-15 19:57:01


In [4]:
print('train_locations shape: ', train_locations.shape)
print('test_locations shape: ', test_locations.shape)
train_locations.head()

train_locations shape:  (59503, 5)
test_locations shape:  (16720, 5)


,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.7,-78.8
1,02SFNJH,1,NaN,1.7,0.8
2,02SFNJH,2,NaN,-0.5,0.7
3,RU43CXC,0,Home,0.1,0.4
4,BDFBPRD,0,NaN,2.5,0.7


In [5]:
print('orders shape: ', orders.shape)
orders.head()

orders shape:  (135303, 26)


,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163238.0,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,105,2019-08-01 05:30:16,0,NaN,92PEE24 X 0 X 105
1,163240.0,QS68UD8,1.0,8.7,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,294,2019-08-01 05:31:10,0,Work,QS68UD8 X 0 X 294
2,163241.0,MB7VY5F,2.0,14.4,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:31:33,0,NaN,MB7VY5F X 0 X 83
3,163244.0,KDJ951Y,1.0,7.1,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,90,2019-08-01 05:34:54,0,Home,KDJ951Y X 0 X 90
4,163245.0,BAL0RVT,4.0,27.2,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:35:51,0,Work,BAL0RVT X 0 X 83


In [6]:
print('vendors shape: ', vendors.shape)
vendors.head()

vendors shape:  (100, 59)


,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,...,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,4,118597.0,-0.6,0.8,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,...,1.0,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1
1,13,118608.0,-0.5,0.7,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,...,1.0,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1
2,20,118616.0,-0.4,0.6,Restaurants,2.0,0.0,8.0,1.0,08:00AM-10:45PM,...,1.0,"4,8,91,10","Breakfast,Desserts,Free Delivery,Indian",Y,1.0,1.0,2018-05-04 22:28:22,2020-04-07 16:35:55,3,1
3,23,118619.0,-0.6,0.8,Restaurants,2.0,0.0,5.0,1.0,10:59AM-10:30PM,...,1.0,"5,8,30,24","Burgers,Desserts,Fries,Salads",Y,1.0,1.0,2018-05-06 19:20:48,2020-04-02 00:56:17,3,1
4,28,118624.0,0.5,0.6,Restaurants,2.0,0.7,15.0,1.0,11:00AM-11:45PM,...,1.0,5,Burgers,Y,1.0,1.0,2018-05-17 22:12:38,2020-04-05 15:57:41,3,1


### On merge les customer et leur localisation

In [7]:
# merge train_customers and train_locations
train = pd.merge(train_customers, train_locations, left_on='akeed_customer_id', right_on='customer_id')
train.drop(['akeed_customer_id'], axis=1, inplace=True)
train['CID X LOC'] = train['customer_id'] + ' X ' + train['location_number'].astype(str)

print('train shape: ', train.shape)
train.head()

train shape:  (58213, 13)


,gender,dob,status,verified,language,created_at,updated_at,customer_id,location_number,location_type,latitude,longitude,CID X LOC
0,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23,TCHWPBT,0,Work,-96.4,-67.2,TCHWPBT X 0
1,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23,TCHWPBT,1,Home,-96.6,-67.2,TCHWPBT X 1
2,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23,TCHWPBT,2,NaN,-0.1,-78.6,TCHWPBT X 2
3,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41,ZGFSYCZ,0,Home,-0.2,-78.6,ZGFSYCZ X 0
4,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41,ZGFSYCZ,1,Home,0.2,-78.6,ZGFSYCZ X 1


In [8]:
# merge test_customers and test_locations
test = pd.merge(test_customers, test_locations, left_on='akeed_customer_id', right_on='customer_id')
test.drop(['akeed_customer_id'], axis=1, inplace=True)
test['CID X LOC'] = test['customer_id'] + ' X ' + test['location_number'].astype(str)

print('test shape: ', test.shape)
test.head()

test shape:  (16333, 13)


,gender,dob,status,verified,language,created_at,updated_at,customer_id,location_number,location_type,latitude,longitude,CID X LOC
0,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,0,NaN,-9.6e+01,-67.2,ICE2DJP X 0
1,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,1,NaN,3.9e-02,-78.6,ICE2DJP X 1
2,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,2,Other,-9.5e+01,43.7,ICE2DJP X 2
3,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,3,Other,-9.6e+01,43.6,ICE2DJP X 3
4,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,4,Work,2.0e+02,44.1,ICE2DJP X 4


### Nettoyage de la donnée

Nettoyage des customer X location

In [9]:
# label encoding
from sklearn.preprocessing import LabelEncoder

# print train columns
print('train columns: ', train.columns)
print('train shape: ', train.shape)
print('number of missing values per column:')
display(train.isnull().sum())
print('df_input types: ', train.dtypes)

def clean_df(df, le):
    # no warning
    pd.options.mode.chained_assignment = None

    # df cleaning
    new_df = df[['CID X LOC', 'latitude', 'longitude', 'location_type', 'status', 'gender', 'language']]

    # we fill the missing values with the mean
    new_df['latitude'].fillna(new_df['latitude'].mean(), inplace=True)
    new_df['longitude'].fillna(new_df['longitude'].mean(), inplace=True)
    new_df['language'].fillna('EN', inplace=True)
    new_df['gender'].fillna('None', inplace=True)
    new_df['location_type'].fillna('None', inplace=True)

    # fit and transform the location_type column
    new_df['location_type'] = le.fit_transform(new_df['location_type'])
    new_df['language'] = le.fit_transform(new_df['language'])
    new_df['gender'] = le.fit_transform(new_df['gender'])

    # set back the warning
    pd.options.mode.chained_assignment = 'warn'

    return new_df


# create a LabelEncoder object
le = LabelEncoder()

train = clean_df(train, le)
test = clean_df(test, le)
train.head()

train columns:  Index(['gender', 'dob', 'status', 'verified', 'language', 'created_at',
       'updated_at', 'customer_id', 'location_number', 'location_type',
       'latitude', 'longitude', 'CID X LOC'],
      dtype='object')
train shape:  (58213, 13)
number of missing values per column:


gender             17051
dob                53018
status                 0
verified               0
language           15769
created_at             0
updated_at             0
customer_id            0
location_number        0
location_type      26593
latitude               6
longitude              6
CID X LOC              0
dtype: int64

df_input types:  gender              object
dob                float64
status               int64
verified             int64
language            object
created_at          object
updated_at          object
customer_id         object
location_number      int64
location_type       object
latitude           float64
longitude          float64
CID X LOC           object
dtype: object


,CID X LOC,latitude,longitude,location_type,status,gender,language
0,TCHWPBT X 0,-96.4,-67.2,3,1,7,0
1,TCHWPBT X 1,-96.6,-67.2,0,1,7,0
2,TCHWPBT X 2,-0.1,-78.6,1,1,7,0
3,ZGFSYCZ X 0,-0.2,-78.6,0,1,7,0
4,ZGFSYCZ X 1,0.2,-78.6,0,1,7,0


Nettoyage des vendors

In [10]:
# print vendors columns
print('vendors columns: ', vendors.columns)
print('vendors shape: ', vendors.shape)
print('number of missing values per column:')
display(vendors.isnull().sum())
print('df_input types: ', vendors.dtypes)

vendors = vendors[['id', 'latitude', 'longitude', 'vendor_category_id', 'delivery_charge', 'serving_distance', 
                   'is_open', 'prepration_time', 'discount_percentage', 'rank', 'language', 'vendor_rating', 'device_type', 'vendor_tag_name']]
# rename the id column to VENDOR
vendors.rename(columns={'id': 'VENDOR'}, inplace=True)


vendors['language'].fillna('EN', inplace=True)
vendors["vendor_tag_name"].fillna("", inplace=True)

vendors['language'] = le.fit_transform(vendors['language'])

vendors.head()

vendors columns:  Index(['id', 'authentication_id', 'latitude', 'longitude',
       'vendor_category_en', 'vendor_category_id', 'delivery_charge',
       'serving_distance', 'is_open', 'OpeningTime', 'OpeningTime2',
       'prepration_time', 'commission', 'is_akeed_delivering',
       'discount_percentage', 'status', 'verified', 'rank', 'language',
       'vendor_rating', 'sunday_from_time1', 'sunday_to_time1',
       'sunday_from_time2', 'sunday_to_time2', 'monday_from_time1',
       'monday_to_time1', 'monday_from_time2', 'monday_to_time2',
       'tuesday_from_time1', 'tuesday_to_time1', 'tuesday_from_time2',
       'tuesday_to_time2', 'wednesday_from_time1', 'wednesday_to_time1',
       'wednesday_from_time2', 'wednesday_to_time2', 'thursday_from_time1',
       'thursday_to_time1', 'thursday_from_time2', 'thursday_to_time2',
       'friday_from_time1', 'friday_to_time1', 'friday_from_time2',
       'friday_to_time2', 'saturday_from_time1', 'saturday_to_time1',
       'saturday_from

id                       0
authentication_id        0
latitude                 0
longitude                0
vendor_category_en       0
vendor_category_id       0
delivery_charge          0
serving_distance         0
is_open                  0
OpeningTime              9
OpeningTime2             9
prepration_time          0
commission              15
is_akeed_delivering      0
discount_percentage      0
status                   0
verified                 0
rank                     0
language                15
vendor_rating            0
sunday_from_time1        1
sunday_to_time1          1
sunday_from_time2       58
sunday_to_time2         58
monday_from_time1        0
monday_to_time1          0
monday_from_time2       58
monday_to_time2         58
tuesday_from_time1       1
tuesday_to_time1         1
tuesday_from_time2      59
tuesday_to_time2        59
wednesday_from_time1     0
wednesday_to_time1       0
wednesday_from_time2    58
wednesday_to_time2      58
thursday_from_time1      1
t

df_input types:  id                        int64
authentication_id       float64
latitude                float64
longitude               float64
vendor_category_en       object
vendor_category_id      float64
delivery_charge         float64
serving_distance        float64
is_open                 float64
OpeningTime              object
OpeningTime2             object
prepration_time           int64
commission              float64
is_akeed_delivering      object
discount_percentage     float64
status                  float64
verified                  int64
rank                      int64
language                 object
vendor_rating           float64
sunday_from_time1        object
sunday_to_time1          object
sunday_from_time2        object
sunday_to_time2          object
monday_from_time1        object
monday_to_time1          object
monday_from_time2        object
monday_to_time2          object
tuesday_from_time1       object
tuesday_to_time1         object
tuesday_from_time2     

,VENDOR,latitude,longitude,vendor_category_id,delivery_charge,serving_distance,is_open,prepration_time,discount_percentage,rank,language,vendor_rating,device_type,vendor_tag_name
0,4,-0.6,0.8,2.0,0.0,6.0,1.0,15,0.0,11,0,4.4,3,"Arabic,Breakfast,Burgers,Desserts,Free Deliver..."
1,13,-0.5,0.7,2.0,0.7,5.0,1.0,14,0.0,11,0,4.7,3,"Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa..."
2,20,-0.4,0.6,2.0,0.0,8.0,1.0,19,0.0,1,0,4.5,3,"Breakfast,Desserts,Free Delivery,Indian"
3,23,-0.6,0.8,2.0,0.0,5.0,1.0,16,0.0,11,0,4.5,3,"Burgers,Desserts,Fries,Salads"
4,28,0.5,0.6,2.0,0.7,15.0,1.0,10,0.0,11,0,4.4,3,Burgers


transform the last column into multiples column for each tag, filled with 0 or 1 if the tag is in the row

In [11]:
# split the vendor_tag_name strings into a list of strings
vendors['vendor_tag_name'] = vendors['vendor_tag_name'].str.split(',')

vendors.head()

,VENDOR,latitude,longitude,vendor_category_id,delivery_charge,serving_distance,is_open,prepration_time,discount_percentage,rank,language,vendor_rating,device_type,vendor_tag_name
0,4,-0.6,0.8,2.0,0.0,6.0,1.0,15,0.0,11,0,4.4,3,"[Arabic, Breakfast, Burgers, Desserts, Free De..."
1,13,-0.5,0.7,2.0,0.7,5.0,1.0,14,0.0,11,0,4.7,3,"[Breakfast, Cakes, Crepes, Italian, Pasta, Piz..."
2,20,-0.4,0.6,2.0,0.0,8.0,1.0,19,0.0,1,0,4.5,3,"[Breakfast, Desserts, Free Delivery, Indian]"
3,23,-0.6,0.8,2.0,0.0,5.0,1.0,16,0.0,11,0,4.5,3,"[Burgers, Desserts, Fries, Salads]"
4,28,0.5,0.6,2.0,0.7,15.0,1.0,10,0.0,11,0,4.4,3,[Burgers]


In [12]:
#create a vector with all the tags in vendor_tag_name with no duplicates
tags = []
for e in vendors['vendor_tag_name']:
    for tag in e:
        tags.append(tag.strip())

tags = list(set(tags))
tags.remove('')

print('number of tags: ', len(tags))
print(tags)

number of tags:  67
['Cafe', 'Sushi', 'Indian', 'Kids meal', 'Waffles', 'Dimsum', 'Rice', 'Steaks', 'Manakeesh', 'Chinese', 'Bagels', 'Mexican', 'Family Meal', 'Pancakes', 'Kebabs', 'Japanese', 'Sweets', 'Crepes', 'Pasta', 'Combos', 'Donuts', 'Breakfast', 'Hot Chocolate', 'Free Delivery', 'American', 'Smoothies', 'Organic', 'Ice creams', 'Mojitos', 'Grills', 'Milkshakes', 'Thai', 'Omani', 'Churros', 'Shawarma', 'Asian', 'Spanish Latte', 'Shuwa', 'Pastry', 'Burgers', 'Pizzas', 'Italian', 'Pizza', 'Pastas', 'Frozen yoghurt', 'Desserts', 'Seafood', 'Kushari', 'Fatayers', 'Healthy Food', 'Fresh Juices', 'Cakes', 'Sandwiches', 'Lebanese', 'Fries', 'Mandazi', 'Coffee', 'Biryani', 'Salads', 'Vegetarian', 'Mishkak', 'Arabic', 'Thali', 'Hot Dogs', 'Rolls', 'Soups', 'Karak']


In [13]:
# add a column for each tag in vendors
# fill the column with 1 if the tag is in vendor_tag_name
for tag in tags:
    vendors[tag] = vendors['vendor_tag_name'].apply(lambda x: 1 if tag in x else 0)

# drop the vendor_tag_name column
vendors.drop(['vendor_tag_name'], axis=1, inplace=True)

vendors.head()

,VENDOR,latitude,longitude,vendor_category_id,delivery_charge,serving_distance,is_open,prepration_time,discount_percentage,rank,...,Biryani,Salads,Vegetarian,Mishkak,Arabic,Thali,Hot Dogs,Rolls,Soups,Karak
0,4,-0.6,0.8,2.0,0.0,6.0,1.0,15,0.0,11,...,0,1,0,0,1,0,0,0,0,0
1,13,-0.5,0.7,2.0,0.7,5.0,1.0,14,0.0,11,...,0,1,0,0,0,0,0,0,1,0
2,20,-0.4,0.6,2.0,0.0,8.0,1.0,19,0.0,1,...,0,0,0,0,0,0,0,0,0,0
3,23,-0.6,0.8,2.0,0.0,5.0,1.0,16,0.0,11,...,0,1,0,0,0,0,0,0,0,0
4,28,0.5,0.6,2.0,0.7,15.0,1.0,10,0.0,11,...,0,0,0,0,0,0,0,0,0,0


### Fusion des deux vecteurs

In [14]:
# merge train and vendors
train['key'] = 0
vendors['key'] = 0
train = pd.merge(train, vendors, on='key')

# drop the key column
train.drop(['key'], axis=1, inplace=True)

train['CID X LOC X VENDOR'] = train['CID X LOC'] + ' X ' + train['VENDOR'].astype(str)
train.drop(['CID X LOC', 'VENDOR'], axis=1, inplace=True)

print('train shape: ', train.shape)
train.head()

train shape:  (5821300, 86)


,latitude_x,longitude_x,location_type,status,gender,language_x,latitude_y,longitude_y,vendor_category_id,delivery_charge,...,Salads,Vegetarian,Mishkak,Arabic,Thali,Hot Dogs,Rolls,Soups,Karak,CID X LOC X VENDOR
0,-96.4,-67.2,3,1,7,0,-0.6,0.8,2.0,0.0,...,1,0,0,1,0,0,0,0,0,TCHWPBT X 0 X 4
1,-96.4,-67.2,3,1,7,0,-0.5,0.7,2.0,0.7,...,1,0,0,0,0,0,0,1,0,TCHWPBT X 0 X 13
2,-96.4,-67.2,3,1,7,0,-0.4,0.6,2.0,0.0,...,0,0,0,0,0,0,0,0,0,TCHWPBT X 0 X 20
3,-96.4,-67.2,3,1,7,0,-0.6,0.8,2.0,0.0,...,1,0,0,0,0,0,0,0,0,TCHWPBT X 0 X 23
4,-96.4,-67.2,3,1,7,0,0.5,0.6,2.0,0.7,...,0,0,0,0,0,0,0,0,0,TCHWPBT X 0 X 28


ajout de la colonne prédiction pour pouvoir ensuite génerer y, ici on decide de mettre 1 en prediction si le client a en effet commandé (si la ligne existe dans orders), 0 sinon

In [15]:
# keep only the CID X LOC X VENDOR in orders
orders = orders[['CID X LOC_NUM X VENDOR']]

# add a prediction column to orders
orders['prediction'] = 1

# merge train and orders
train['CID X LOC X VENDOR'] = train['CID X LOC X VENDOR'].astype(str)
orders['CID X LOC X VENDOR'] = orders['CID X LOC_NUM X VENDOR'].astype(str)
orders.drop(['CID X LOC_NUM X VENDOR'], axis=1, inplace=True)
train = pd.merge(train, orders, on='CID X LOC X VENDOR', how='left')

# fill the missing values with 0
train['prediction'].fillna(0, inplace=True)

print('train shape: ', train.shape)
train.head()

train shape:  (5875133, 87)


,latitude_x,longitude_x,location_type,status,gender,language_x,latitude_y,longitude_y,vendor_category_id,delivery_charge,...,Vegetarian,Mishkak,Arabic,Thali,Hot Dogs,Rolls,Soups,Karak,CID X LOC X VENDOR,prediction
0,-96.4,-67.2,3,1,7,0,-0.6,0.8,2.0,0.0,...,0,0,1,0,0,0,0,0,TCHWPBT X 0 X 4,0.0
1,-96.4,-67.2,3,1,7,0,-0.5,0.7,2.0,0.7,...,0,0,0,0,0,0,1,0,TCHWPBT X 0 X 13,0.0
2,-96.4,-67.2,3,1,7,0,-0.4,0.6,2.0,0.0,...,0,0,0,0,0,0,0,0,TCHWPBT X 0 X 20,0.0
3,-96.4,-67.2,3,1,7,0,-0.6,0.8,2.0,0.0,...,0,0,0,0,0,0,0,0,TCHWPBT X 0 X 23,0.0
4,-96.4,-67.2,3,1,7,0,0.5,0.6,2.0,0.7,...,0,0,0,0,0,0,0,0,TCHWPBT X 0 X 28,0.0


In [16]:
from math import radians, cos, sin, asin, sqrt

# Haversine formula to calculate distance between two points
def haversine(lat1, lon1, lat2, lon2):
    R = 6372.8 # Earth radius in kilometers
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))
    return R*c

In [17]:
for col in train.columns:
    print(col, train[col].isnull().sum())

# clean the train dataframe

# add a column "same_language" then drop the language_x and language_y column
train['same_language'] = train['language_x'] == train['language_y']
train.drop(['language_x', 'language_y'], axis=1, inplace=True)

# add a column "is_in_delivery_zone" then drop the latitude_x, longitude_x, latitude_y, longitude_y and serving_distance column
train['is_in_delivery_zone'] = train.apply(lambda x: haversine(x['latitude_x'], x['longitude_x'], x['latitude_y'], x['longitude_y']) <= x['serving_distance'], axis=1)
train.drop(['latitude_x', 'longitude_x', 'latitude_y', 'longitude_y', 'serving_distance'], axis=1, inplace=True)

# convert same_language and is_in_delivery_zone to int
train['same_language'] = train['same_language'].astype(int)
train['is_in_delivery_zone'] = train['is_in_delivery_zone'].astype(int)


latitude_x 0
longitude_x 0
location_type 0
status 0
gender 0
language_x 0
latitude_y 0
longitude_y 0
vendor_category_id 0
delivery_charge 0
serving_distance 0
is_open 0
prepration_time 0
discount_percentage 0
rank 0
language_y 0
vendor_rating 0
device_type 0
Cafe 0
Sushi 0
Indian 0
Kids meal 0
Waffles 0
Dimsum 0
Rice 0
Steaks 0
Manakeesh 0
Chinese 0
Bagels 0
Mexican 0
Family Meal 0
Pancakes 0
Kebabs 0
Japanese 0
Sweets 0
Crepes 0
Pasta 0
Combos 0
Donuts 0
Breakfast 0
Hot Chocolate 0
Free Delivery 0
American 0
Smoothies 0
Organic 0
Ice creams 0
Mojitos 0
Grills 0
Milkshakes 0
Thai 0
Omani 0
Churros 0
Shawarma 0
Asian 0
Spanish Latte 0
Shuwa 0
Pastry 0
Burgers 0
Pizzas 0
Italian 0
Pizza 0
Pastas 0
Frozen yoghurt 0
Desserts 0
Seafood 0
Kushari 0
Fatayers 0
Healthy Food 0
Fresh Juices 0
Cakes 0
Sandwiches 0
Lebanese 0
Fries 0
Mandazi 0
Coffee 0
Biryani 0
Salads 0
Vegetarian 0
Mishkak 0
Arabic 0
Thali 0
Hot Dogs 0
Rolls 0
Soups 0
Karak 0
CID X LOC X VENDOR 0
prediction 0


In [18]:
for col in train.columns:
    print(col, train[col].isnull().sum())

# drop CID X LOC X VENDOR column
train.drop(['CID X LOC X VENDOR'], axis=1, inplace=True)

location_type 0
status 0
gender 0
vendor_category_id 0
delivery_charge 0
is_open 0
prepration_time 0
discount_percentage 0
rank 0
vendor_rating 0
device_type 0
Cafe 0
Sushi 0
Indian 0
Kids meal 0
Waffles 0
Dimsum 0
Rice 0
Steaks 0
Manakeesh 0
Chinese 0
Bagels 0
Mexican 0
Family Meal 0
Pancakes 0
Kebabs 0
Japanese 0
Sweets 0
Crepes 0
Pasta 0
Combos 0
Donuts 0
Breakfast 0
Hot Chocolate 0
Free Delivery 0
American 0
Smoothies 0
Organic 0
Ice creams 0
Mojitos 0
Grills 0
Milkshakes 0
Thai 0
Omani 0
Churros 0
Shawarma 0
Asian 0
Spanish Latte 0
Shuwa 0
Pastry 0
Burgers 0
Pizzas 0
Italian 0
Pizza 0
Pastas 0
Frozen yoghurt 0
Desserts 0
Seafood 0
Kushari 0
Fatayers 0
Healthy Food 0
Fresh Juices 0
Cakes 0
Sandwiches 0
Lebanese 0
Fries 0
Mandazi 0
Coffee 0
Biryani 0
Salads 0
Vegetarian 0
Mishkak 0
Arabic 0
Thali 0
Hot Dogs 0
Rolls 0
Soups 0
Karak 0
CID X LOC X VENDOR 0
prediction 0
same_language 0
is_in_delivery_zone 0


### Création des data splits

In [19]:
# creation of y_train
y_train = train['prediction']
train.drop(['prediction'], axis=1, inplace=True)
print('y_train shape: ', y_train.shape)

y_train shape:  (5875133,)


On veut un nombre égaux d'exemple dans les 2 classes

In [20]:
# Count the number of 0 and 1 in y
print(pd.Series(y_train).value_counts())

0.0    5742907
1.0     132226
Name: prediction, dtype: int64


In [21]:
from imblearn.under_sampling import RandomUnderSampler
# Use RandomUnderSampler to balance the classes
rus = RandomUnderSampler(random_state=42)
x_train, y_train = rus.fit_resample(train, y_train)
del train

# Print the class distribution of the balanced y
print(pd.Series(y_train).value_counts())

0.0    132226
1.0    132226
Name: prediction, dtype: int64


In [22]:
# data split
X_train, X_test = train_test_split(x_train, test_size=0.2, random_state=42, shuffle=True)
print('test shape: ', X_test.shape)
print('train shape: ', X_train.shape)

Y_train, Y_test = train_test_split(y_train, test_size=0.2, random_state=42, shuffle=True)

test shape:  (52891, 80)
train shape:  (211561, 80)


### Creation du modèle

In [23]:
# creation of an ai model
# it takes as input a dataframe with the columns as in df_input
# it returns a dataframe with the columns as in all_pairs and a column PREDICTION with the prediction

from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization

model = keras.Sequential()
model.add(Input(shape=(x_train.shape[1],)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               20736     
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 61,953
Trainable params: 61,953
Non-trainable params: 0
_________________________________________________________________


2023-05-14 16:06:36.937919: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-14 16:06:37.224649: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [24]:
# Use LRReducerOnPlateau to reduce learning rate when the validation loss stops improving
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", patience=3, factor=0.2, min_lr=1e-7)

# Create an early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(X_train,
          Y_train,
          epochs=50,
          batch_size=64,
          validation_data=(X_test, Y_test),
          callbacks=[lr_reducer, early_stopping]
)

Epoch 1/50
3306/3306 [==============================] - 3s 821us/step - loss: 0.2232 - accuracy: 0.6316 - val_loss: 0.2199 - val_accuracy: 0.6423 - lr: 0.0010
Epoch 2/50
3306/3306 [==============================] - 3s 810us/step - loss: 0.2173 - accuracy: 0.6502 - val_loss: 0.2146 - val_accuracy: 0.6568 - lr: 0.0010
Epoch 3/50
3306/3306 [==============================] - 3s 797us/step - loss: 0.2157 - accuracy: 0.6527 - val_loss: 0.2146 - val_accuracy: 0.6551 - lr: 0.0010
Epoch 4/50
3306/3306 [==============================] - 3s 789us/step - loss: 0.2144 - accuracy: 0.6567 - val_loss: 0.2141 - val_accuracy: 0.6580 - lr: 0.0010
Epoch 5/50
3306/3306 [==============================] - 3s 800us/step - loss: 0.2138 - accuracy: 0.6572 - val_loss: 0.2140 - val_accuracy: 0.6570 - lr: 0.0010
Epoch 6/50
3306/3306 [==============================] - 3s 791us/step - loss: 0.2133 - accuracy: 0.6586 - val_loss: 0.2136 - val_accuracy: 0.6539 - lr: 0.0010
Epoch 7/50
3306/3306 [========================

In [25]:
# save the model
model.save('model.h5')

### Application sur le jeu de test

preprocessing des données de test comme avec train precedemment

In [26]:
# create a x_test and y_test dataframe using the test dataset
# merge test and vendors
test['key'] = 0
vendors['key'] = 0
test = pd.merge(test, vendors, on='key')

# drop the key column
test.drop(['key'], axis=1, inplace=True)

test['CID X LOC X VENDOR'] = test['CID X LOC'] + ' X ' + test['VENDOR'].astype(str)
test.drop(['CID X LOC', 'VENDOR'], axis=1, inplace=True)

print('train shape: ', test.shape)
test.head()

train shape:  (1633300, 86)


,latitude_x,longitude_x,location_type,status,gender,language_x,latitude_y,longitude_y,vendor_category_id,delivery_charge,...,Salads,Vegetarian,Mishkak,Arabic,Thali,Hot Dogs,Rolls,Soups,Karak,CID X LOC X VENDOR
0,-96.4,-67.2,1,1,3,0,-0.6,0.8,2.0,0.0,...,1,0,0,1,0,0,0,0,0,ICE2DJP X 0 X 4
1,-96.4,-67.2,1,1,3,0,-0.5,0.7,2.0,0.7,...,1,0,0,0,0,0,0,1,0,ICE2DJP X 0 X 13
2,-96.4,-67.2,1,1,3,0,-0.4,0.6,2.0,0.0,...,0,0,0,0,0,0,0,0,0,ICE2DJP X 0 X 20
3,-96.4,-67.2,1,1,3,0,-0.6,0.8,2.0,0.0,...,1,0,0,0,0,0,0,0,0,ICE2DJP X 0 X 23
4,-96.4,-67.2,1,1,3,0,0.5,0.6,2.0,0.7,...,0,0,0,0,0,0,0,0,0,ICE2DJP X 0 X 28


In [27]:
for col in test.columns:
    print(col, test[col].isnull().sum())

# clean the train dataframe

# add a column "same_language" then drop the language_x and language_y column
test['same_language'] = test['language_x'] == test['language_y']
test.drop(['language_x', 'language_y'], axis=1, inplace=True)

# add a column "is_in_delivery_zone" then drop the latitude_x, longitude_x, latitude_y, longitude_y and serving_distance column
test['is_in_delivery_zone'] = test.apply(lambda x: haversine(x['latitude_x'], x['longitude_x'], x['latitude_y'], x['longitude_y']) <= x['serving_distance'], axis=1)
test.drop(['latitude_x', 'longitude_x', 'latitude_y', 'longitude_y', 'serving_distance'], axis=1, inplace=True)

# convert same_language and is_in_delivery_zone to int
test['same_language'] = test['same_language'].astype(int)
test['is_in_delivery_zone'] = test['is_in_delivery_zone'].astype(int)

latitude_x 0
longitude_x 0
location_type 0
status 0
gender 0
language_x 0
latitude_y 0
longitude_y 0
vendor_category_id 0
delivery_charge 0
serving_distance 0
is_open 0
prepration_time 0
discount_percentage 0
rank 0
language_y 0
vendor_rating 0
device_type 0
Cafe 0
Sushi 0
Indian 0
Kids meal 0
Waffles 0
Dimsum 0
Rice 0
Steaks 0
Manakeesh 0
Chinese 0
Bagels 0
Mexican 0
Family Meal 0
Pancakes 0
Kebabs 0
Japanese 0
Sweets 0
Crepes 0
Pasta 0
Combos 0
Donuts 0
Breakfast 0
Hot Chocolate 0
Free Delivery 0
American 0
Smoothies 0
Organic 0
Ice creams 0
Mojitos 0
Grills 0
Milkshakes 0
Thai 0
Omani 0
Churros 0
Shawarma 0
Asian 0
Spanish Latte 0
Shuwa 0
Pastry 0
Burgers 0
Pizzas 0
Italian 0
Pizza 0
Pastas 0
Frozen yoghurt 0
Desserts 0
Seafood 0
Kushari 0
Fatayers 0
Healthy Food 0
Fresh Juices 0
Cakes 0
Sandwiches 0
Lebanese 0
Fries 0
Mandazi 0
Coffee 0
Biryani 0
Salads 0
Vegetarian 0
Mishkak 0
Arabic 0
Thali 0
Hot Dogs 0
Rolls 0
Soups 0
Karak 0
CID X LOC X VENDOR 0


In [28]:
for col in test.columns:
    print(col, test[col].isnull().sum())

# keep the CID X LOC X VENDOR column in a vector
CID_X_LOC_X_VENDOR = test['CID X LOC X VENDOR']

# drop CID X LOC X VENDOR column
test.drop(['CID X LOC X VENDOR'], axis=1, inplace=True)

location_type 0
status 0
gender 0
vendor_category_id 0
delivery_charge 0
is_open 0
prepration_time 0
discount_percentage 0
rank 0
vendor_rating 0
device_type 0
Cafe 0
Sushi 0
Indian 0
Kids meal 0
Waffles 0
Dimsum 0
Rice 0
Steaks 0
Manakeesh 0
Chinese 0
Bagels 0
Mexican 0
Family Meal 0
Pancakes 0
Kebabs 0
Japanese 0
Sweets 0
Crepes 0
Pasta 0
Combos 0
Donuts 0
Breakfast 0
Hot Chocolate 0
Free Delivery 0
American 0
Smoothies 0
Organic 0
Ice creams 0
Mojitos 0
Grills 0
Milkshakes 0
Thai 0
Omani 0
Churros 0
Shawarma 0
Asian 0
Spanish Latte 0
Shuwa 0
Pastry 0
Burgers 0
Pizzas 0
Italian 0
Pizza 0
Pastas 0
Frozen yoghurt 0
Desserts 0
Seafood 0
Kushari 0
Fatayers 0
Healthy Food 0
Fresh Juices 0
Cakes 0
Sandwiches 0
Lebanese 0
Fries 0
Mandazi 0
Coffee 0
Biryani 0
Salads 0
Vegetarian 0
Mishkak 0
Arabic 0
Thali 0
Hot Dogs 0
Rolls 0
Soups 0
Karak 0
CID X LOC X VENDOR 0
same_language 0
is_in_delivery_zone 0


test du modèle avec le jeu de test et création du fichier Submission.csv

In [29]:
# if needed
model = keras.models.load_model('model.h5')

In [35]:
# predict the test dataset
y_pred = model.predict(test)

51041/51041 [==============================] - 21s 410us/step


In [38]:
# flatten the y_pred
y_pred = y_pred.flatten()

# convert the y_pred to int (0 or 1)
y_pred = y_pred.round().astype(int)

In [39]:
# create the submission dataframe
submission = pd.DataFrame({'CID X LOC X VENDOR': CID_X_LOC_X_VENDOR, 'target': y_pred.flatten()})

# save the submission dataframe
submission.to_csv('Submission.csv', index=False)